In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import random
import glob
import csv

## Initialize Models
### Define constants
Define the timesteps for the recurrent neural network. The timesteps is equal the number of pitches in the pitching sequence that you want the AI to analyze at a time.

In [ ]:
TIMESTEPS = 5

### Recurrent neural network
The recurrent neural network used in this project is a simple [long short-term memory (LSTM)](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) network

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(256, return_sequences=False, input_shape=(TIMESTEPS, 21)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

### Configure checkpoints
We will use TensorFlow's [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the weights of the model at each epoch so that we select the weights from the best epoch to use for the final model once training is complete.

In [ ]:
cp_filepath = 'pitch_checkpoints/cp-{epoch:04d}.ckpt'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=cp_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=False)

## Load data
Each plate appearance by the opponent has it's own csv associated with it which contains the pitching information against the oponent. For example, when a new oponent steps up to the plate, a new csv will be created and will record data against that batter until that batter gets a hit or get out. The csv is formatted as `outcome,pitch_type,pitch_direction,strikes,balls,batter_handedness`. The `outcome` of the pitch is treated as its label, and the rest of the fields are treated as the input data for the model. All of the input fields will be converted to be [one hot encoded](https://en.wikipedia.org/wiki/One-hot). The first field of the one hot encoding will represent blank data. This is because the AI will consider the last n amount of pitches against a batter, where n is defined by `TIMESTEPS`. However, when throwing the first pitch against a new batter, there will be no previous pitch data to look at, so we need a way to represent this as being empty. So the one hot encoding for the `pitch_type` field will be like `[empty, fastball, screwball, curveball, splitter]`. A curveball pitch in this case would be represented as `[0, 0, 0, 1, 0]`.

In [ ]:
# variables for converting from label encoding to one hot encoding
pitch_types = {
    'fastball': 1,
    'screwball': 2,
    'curveball': 3,
    'splitter': 4
}

hands = {
    'left': 1,
    'right': 2
}

directions = {
    'left': 1,
    'center': 2,
    'right': 3
}

CSV_DIR = '../data/examples/pitching/csvs_pitches/'
csv_paths = glob.glob(f'{CSV_DIR}*.csv')
random.shuffle(csv_paths)

data = []
labels = []

# re ad each csv line by line
for csv_path in csv_paths:
    with open(csv_path, newline='') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        # initialize the pitch sequence represent all empty data
        sequence = []
        for i in range(TIMESTEPS):
            hand = [1, 0, 0]
            pitch_type = [1, 0, 0, 0, 0]
            direction = [1, 0, 0, 0]
            strikes = [1, 0, 0, 0]
            balls = [1, 0, 0, 0, 0]
            pitch = [hand, pitch_type, direction, strikes, balls]
            pitch = [item for sublist in pitch for item in sublist]
            sequence.append(np.array(pitch))
        
        # one hot encoding
        for row in reader:
            hand = [0, 0, 0]
            hand[hands[row[5]]] = 1
            
            pitch_type = [0, 0, 0, 0, 0]
            pitch_type[pitch_types[row[1]]] = 1
            
            direction = [0, 0, 0, 0]
            direction[directions[row[2]]] = 1
            
            strikes = [0, 0, 0, 0]
            strikes[int(row[3])+1] = 1
            
            balls = [0, 0, 0, 0, 0]
            balls[int(row[4])+1] = 1
            
            pitch = [hand, pitch_type, direction, strikes, balls]
            pitch = [item for sublist in pitch for item in sublist]
            
            # delete first pitch in the current sequence and append the new pitch to the sequence
            del sequence[0]
            sequence.append(np.array(pitch))
            
            data.append(np.array(sequence))
            labels.append(float(row[0]))           

data = np.array(data)
labels = np.array(labels)

print(data.shape)
print(labels.shape)

## Train model
Train the model on the data collected. You can adjust the `BATCH_SIZE`, `EPOCHS`, and `VALIDATION_SPLIT` paramaters as you like.

In [ ]:
BATCH_SIZE = 256
EPOCHS = 20
VALIDATION_SPLIT = 0.2

history = model.fit(
    x=data,
    y=labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks=[model_checkpoint_callback]
)

## Evaluate model
Evaluate the loss of the model to determine the best checkpoint to load the weights from. The lower the loss, the better the model should be at correctly predicting the outcome of a given pitch.

In [ ]:
# Retrieve a list of loss results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
print(np.argmin(np.array(val_loss)))

# Get range of epochs
epochs_range = range(EPOCHS)

# Plot training and validation loss per epoch
plt.plot(epochs_range, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.legend()

## Save model
Set the `best_cp` variable to the epoch that had the best metrics and save the model as an [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) file.

In [ ]:
best_cp = 18
best_cp_filepath = cp_filepath.format(epoch=best_cp)
model.load_weights(best_cp_filepath)
model.save('pitch_model.h5')

## Inference
In order to use the model for inference and selecting the best pitch you will need to predict the output for each pitch for the given game state, and select the one which the model predicts to have the highest outcome. Here is an example on some dummy data

In [ ]:
# initialize the sequence to represent empty data
sequence = []
for i in range(TIMESTEPS-1):
    sequence.append([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

# test game state data
hand = [0, 0, 1]
strikes = [0, 1, 0, 0]
balls = [0, 1, 0, 0, 0]

best_reward = -1000
best_pitch = ''
# predict outcome for each type of pitch
for direction in ['left', 'center', 'right']:
    for pitch_type in ['fastball', 'screwball', 'curveball', 'splitter']:
        pitch_type_arr = [0, 0, 0, 0, 0]
        pitch_type_arr[pitch_types[pitch_type]] = 1

        direction_arr = [0, 0, 0, 0]
        direction_arr[directions[direction]] = 1

        pitch = [hand, pitch_type_arr, direction_arr, strikes, balls]
        pitch = [item for sublist in pitch for item in sublist]
        
        cur_sequence = list(sequence)
        cur_sequence.append(np.array(pitch))
        
        test_data = np.array([cur_sequence])
        prediction = model(test_data).numpy()[0][0]
        
        if prediction > best_reward:
            best_reward = prediction
            best_pitch = f'{direction} {pitch_type}: {prediction}'
        
        print(f'{direction} {pitch_type}: {prediction}')

print('---------------------')
print(f'Best pitch: {best_pitch}')